In [0]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [0]:
## Il y a 2 mois, le 1er du mois
two_month_ago_start_date = date.today().replace(day=1) - relativedelta(months=-2)

In [0]:
df_trips_cleansed= spark.read.table("nyctaxi.02_silver.yellow_trips_cleansed").filter(f"tpep_pickup_datetime >= '{two_month_ago_start_date}'")
# display(df_trips_cleansed)

In [0]:
df_zone=spark.read.table("nyctaxi.02_silver.taxi_zone_lookup")
# display(df_zone)

In [0]:
df_trips_with_pu = (
    df_trips_cleansed.join(
        df_zone,
        on=df_trips_cleansed.pu_location_id == df_zone.location_id,
        how="left"
    )
).select(
    df_trips_cleansed["*"],
    df_zone.Borough.alias("pu_borough"),
    df_zone.zone.alias("pu_zone"),
)

In [0]:
df_trips_enriched = (
    df_trips_with_pu.join(
        df_zone,
        on=df_trips_cleansed.do_location_id == df_zone.location_id,
        how="left"
    )
).select(
    df_trips_with_pu["*"],
    df_zone.Borough.alias("do_borough"),
    df_zone.zone.alias("do_zone"),
).drop("pu_location_id","do_location_id")

In [0]:
# display(df_trips_enriched)


In [0]:
df_trips_enriched.write.mode("append").saveAsTable("nyctaxi.02_silver.yellow_trips_enriched")

In [0]:
# spark.read.table("nyctaxi.02_silver.yellow_trips_enriched").display()